This notebook helps to choose best model to train on.

In [1]:
import sys
sys.path.append("../asmk")
%ls

conda_requirements.txt  LICENSE                        setup.py
core/                   MANIFEST.in*                   timm_models.ipynb
experiments/            README.md                      timm_models.log
image_retrieval/        scripts/                       _version.py
__init__.py*            search_timm_models_resnet.csv


In [2]:
# General
import argparse
from random import sample
import shutil
import numpy as np
from tqdm import tqdm
import csv
import logging

# Torch
import torch
import torch.nn as nn
import torch.optim as optim

# Timm
import timm 

# Core
from core.utils.configurations              import make_config, config_to_string

# Image Retrieval
from image_retrieval.utils.io               import create_experiment_file_from_cfg , create_experiment_file
from image_retrieval.configuration          import DEFAULTS as DEFAULT_CONFIGS
from image_retrieval.tools                  import make_dataloader, make_model, compute_PCA_layer, get_data_sample
from image_retrieval.tools                  import train, validate, test
from image_retrieval.modules.heads.head     import globalHead
from image_retrieval.models.GF_net          import ImageRetrievalNet

/home/dl/miniconda3/envs/loc/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Parameters
class Parameters:
    config_path     = "./image_retrieval/configuration/defaults/search_timm_models.ini"
    directory       = "./experiments/"
    data            = "/media/dl/Data/datasets/"
    models_family   = "resnet"

params = Parameters() 

In [4]:
# Logging
import logging
logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

In [5]:
# Initialize multi-processing
rank, world_size = 0, 1
device = torch.device(0)

# Set device
torch.cuda.set_device(device)

# Set seed
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)
np.random.seed(0)
    
# Load configuration
cfg = make_config(config_path=params.config_path, 
                  defauls=DEFAULT_CONFIGS["empty"],
                  logger=logger)

# Experiment Path, from cfg
exp_dir = create_experiment_file(params.directory, extension="search_timm_model", logger=logger)

# Initialize logging
logger.info("\n%s", config_to_string(cfg))


# Load data
train_dl, _ = make_dataloader(params, cfg, rank, world_size,
                                          logger=logger)
    
# Sample data 
sample_dl =  get_data_sample(cfg, train_dl, logger=logger)


# Get timm models list
timm_model_list = timm.list_models(pretrained=True, filter=params.models_family+"*")
logger.info("%s %s model found ", len(timm_model_list), params.models_family)
    
# CSV file 
fileds = ['Model', 'Dim', 'E', 'M', 'H',]
    
with open("search_timm_models_" + params.models_family +".csv", 'w') as file:
    writer = csv.DictWriter(file, fieldnames=fileds)
    writer.writeheader()
        
    for model_name in timm_model_list:
            
        # parse params with default values
        body_cfg     = cfg["body"]
        global_cfg   = cfg["global"]
            
        body_cfg["arch"]              = model_name
        body_cfg["features_scales"]   = str([1, 2, 3, 4])

            
        # Create backbone            
        logger.info("Creating backbone model: %s  with features_scales: %s",  model_name,
                                                                                str(body_cfg["features_scales"]))
            
        # Load model state dictionary
        body = timm.create_model(model_name, 
                                    features_only=True, 
                                    out_indices=body_cfg.getstruct("features_scales"), 
                                    pretrained=True,
                                    )
            
        body_channels           = body.feature_info.channels()
        body_reductions         = body.feature_info.reduction()
        body_module_names       = body.feature_info.module_name()
            
        logger.debug("Body channels: %s    Reductions: %s      Layer_names: %s",   body_channels, 
                                                                                    body_reductions,
                                                                                    body_module_names)
            
        # model redunction
        body_dim = body_channels[-1]
        if global_cfg.getboolean("reduction"):
            out_dim = global_cfg.getint("global_dim")
        else:
            out_dim =  body_dim 
            
        # Head
        global_head = globalHead(   inp_dim=body_dim,
                                        global_dim=out_dim,
                                        local_dim=128,
                                        
                                        pooling=global_cfg.getstruct("pooling"),
                                        do_withening=global_cfg.getboolean("whithening"),
                                        layer=global_cfg.get("type"),
                                        )
        # freeze head 
        if global_cfg.getboolean("freeze_whiten"):
            logger.debug("Freeze withening  paramters")
            for p in global_head.whiten.parameters():
                p.requires_grad = False
                
            for p in global_head.local_whiten.parameters():
                p.requires_grad = False
            
        # Create a generic image retrieval network
        model = ImageRetrievalNet(body, global_head)  
            
            
        # Compute PCA
        compute_PCA_layer(model, sample_dl, params, cfg,
                              device=device, 
                              logger=logger)
                
            # # Load model, and init PCA layer
            # model, out_dim = make_model(args, config, train_dataloader, 
            #                                 rank=rank, 
            #                                 world_size=world_size, 
            #                                 device=device,
            #                                 log_debug=log_debug,
            #                                 log_info=log_info)

        # Init GPU stuff
        model = model.cuda(device)
            
        # Eval
        logger.info("Evaluate model { %s }" , model_name)

        scores , _ = test(params, cfg, model, 
                            train_imgs=train_dl.dataset.images,
                            rank=rank, 
                            world_size=world_size,
                            out_dim=out_dim,
                            device=device,
                            logger=logger)


        scores["Model"] = model_name
        scores["Dim"]   = out_dim
            
        # Write to csv
        writer.writerow(scores)
    
logger.info("Evaluation Done ..... ")

2022-08-29 18:29:54,525 | INFO : Loading configuration from ./image_retrieval/configuration/defaults/search_timm_models.ini
2022-08-29 18:29:54,527 | INFO : 
[general]
gpu_id = 0

[body]
arch = resnet50
pretrained = True
features_scales = [1, 2, 3, 4]

[global]
pooling = {"name": "GeM", "params": {"p":3, "eps": 1e-6}}
type = linear
whithening = True
reduction = False
num_samples = 5000
update = True

[dataloader]
dataset = retrieval-SfM-120k
neg_num = 5
query_size = 2000
pool_size = 20000
min_size = 200
max_size = 1024
batch_size = 5
num_workers = 14

[augmentaion]
use_prefetcher = False
no_aug = False
scale = (0.8, 1.2)
ratio = (0.75, 1.33)
hflip = 0.4
vflip = 0.4
color_jitter = 0.4
auto_augment = rand-m5-n3-mstd0.2
interpolation = bilinear
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
re_prob = 0.25
re_mode = pixel
re_count = 1
re_num_splits = 0
crop_pct = None
tf_preprocessing = False
separate = True

[test]
datasets = ["roxford5k"]
batch_size = 1
min_size = 200
max_size 

100%|██████████| 5000/5000 [00:39<00:00, 126.16it/s]

2022-08-29 18:30:36,886 | DEBUG : Compute PCA, Takes a while


2022-08-29 18:30:37,149 | DEBUG : Load : ./experiments/whithen/global_retrieval-SfM-120k_resnet18_L4_DNone_Size1024.pth 
2022-08-29 18:30:37,151 | INFO : Evaluate model { resnet18 }
2022-08-29 18:30:37,152 | DEBUG : Evaluating network on test datasets { global_descriptor }
2022-08-29 18:30:37,153 | DEBUG : Test scales {[1.0]}
2022-08-29 18:30:37,153 | DEBUG : {roxford5k}: Loading Dataset
2022-08-29 18:30:37,161 | DEBUG : {roxford5k}: Extracting descriptors for query images


100%|██████████| 70/70 [00:00<00:00, 124.65it/s]

...

2022-08-29 18:32:09,052 | DEBUG : Load : ./experiments/whithen/global_retrieval-SfM-120k_resnet18d_L4_DNone_Size1024.pth 
2022-08-29 18:32:09,054 | INFO : Evaluate model { resnet18d }
2022-08-29 18:32:09,055 | DEBUG : Evaluating network on test datasets { global_descriptor }
2022-08-29 18:32:09,055 | DEBUG : Test scales {[1.0]}
2022-08-29 18:32:09,056 | DEBUG : {roxford5k}: Loading Dataset
2022-08-29 18:32:09,061 | DEBUG : {roxford5k}: Extracting descriptors for query images


100%|██████████| 70/70 [00:00<00:00, 120.93it/s]

2022-08-29 18:32:10,386 | DEBUG : {roxford5k}: Extracting descriptors for database images



  3%|▎         | 142/4993 [00:01<00:49, 97.07it/s] 


KeyboardInterrupt: 